# Oasis

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import time
from dfply import *

In [2]:
import psycopg2

conn = psycopg2.connect(
    host = "localhost",
    user = "postgres",
    password="June3060!!",
    database = "mimic_data",
    port="5432"
)

cursor = conn.cursor()
print("database connected")

database connected


In [3]:
df = pd.read_csv("../Data/reintubation_df_240127.csv", index_col="Unnamed: 0")

In [4]:
df2 = df.copy()

------

In [5]:
start_time = time.time()

cursor = conn.cursor()
cursor.execute("select * from mimiciv_derived.oasis;")
rows = cursor.fetchall()
columns_name = [desc[0] for desc in cursor.description]
oasis = pd.DataFrame(rows, columns=columns_name)

end_time = time.time()
wait_time = end_time - start_time
print("경과 시간 : ", wait_time)
oasis

경과 시간 :  0.5586652755737305


,subject_id,hadm_id,stay_id,oasis,oasis_prob,age,age_score,preiculos,preiculos_score,gcs,...,resprate,resp_rate_score,temp,temp_score,urineoutput,urineoutput_score,mechvent,mechvent_score,electivesurgery,electivesurgery_score
0,15726063,23069026,36415879,36,0.17014499605424941551,53.3839872681065644,6,73.0000000000000000,3,15.0,...,31.0,6.0,38.56,2.0,4580.0,0.0,1,9,0,6
1,14304873,20156152,35621403,28,0.06884295828308623481,75.1505757369999434,6,9241.9666666666666667,1,14.0,...,32.0,6.0,35.40,4.0,2390.0,1.0,0,0,0,6
2,12117555,25128945,37212487,34,0.13709860673818602936,82.37789823064038515712,9,14055.783333333333,1,9.0,...,27.0,1.0,36.6150000000000000,0.0,85.0,10.0,0,0,0,6
3,16645325,26608451,37532286,33,0.12270071663757324864,77.62581604951116124530,9,1691.9833333333333333,1,15.0,...,24.0,1.0,38.00,2.0,1015.0,5.0,0,0,0,6
4,10989579,26999802,38267080,26,0.05418719720382744723,25.87166522470033566786,3,95.0000000000000000,3,15.0,...,34.0,6.0,36.39,2.0,1225.0,5.0,0,0,0,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73176,16312808,28382354,37548942,13,0.01080267545727155188,39.94355629661673325874,3,1130.9166666666666667,0,15.0,...,24.0,1.0,39.61,2.0,3575.0,0.0,0,0,0,6
73177,14472165,20369424,38998125,33,0.12270071663757324864,57.95088210921343474555,6,4255.2333333333333333,1,14.0,...,44.0,6.0,36.22,2.0,2670.0,0.0,0,0,0,6
73178,19603646,24079072,35813245,28,0.06884295828308623481,47.62103231099745739355,3,5557.1833333333333333,1,15.0,...,26.0,1.0,36.39,2.0,1125.0,5.0,1,9,0,6
73179,16915170,20987661,38612688,20,0.02596747254482635520,47.84390204911325154890,3,23.0000000000000000,3,13.0,...,17.5,0.0,36.94,2.0,1500.0,1.0,0,0,0,6


In [13]:
oasis.columns

Index(['subject_id', 'hadm_id', 'stay_id', 'oasis', 'oasis_prob', 'age',
       'age_score', 'preiculos', 'preiculos_score', 'gcs', 'gcs_score',
       'heartrate', 'heart_rate_score', 'meanbp', 'mbp_score', 'resprate',
       'resp_rate_score', 'temp', 'temp_score', 'urineoutput',
       'urineoutput_score', 'mechvent', 'mechvent_score', 'electivesurgery',
       'electivesurgery_score'],
      dtype='object')

In [7]:
'''
기본 골자
oasis 데이터에 subject_id, hadm_id, stay_id 존재
paired_df의 subject_id, hadm_id, (int_stayid or ext_stayid) 값과 동일한 데이터를 병합
'''


def group_df_oasis(group):

    # group의 index, row를 하나씩 할당
    for idx, row in group.iterrows():

        # oasis의 subject_id, hadm_id가 row의 subject_id, hadm_id와 동일한 oasis_df를 만든다.
        oasis_df = oasis[(oasis["subject_id"] == row["subject_id"]) & (oasis["hadm_id"] == row["hadm_id"])]

        # 만약 row의 int_stayid가 null 값이 아니라면
        if pd.notnull(row["int_stayid"]):

            # oasis_df의 stay_id가 row의 int_stayid를 만족하는 index를 찾는다
            # oasis 데이터 프레임에서 stay_id는 중복된 값이 없고, 전부 unique 값을 갖는다.
            oasis_idx = oasis_df[oasis_df["stay_id"] == row["int_stayid"]].first_valid_index()

            # group의 해당 index에 할당
            group.loc[idx, "oasis"] = oasis_df.loc[oasis_idx, "oasis"]

        # 만약 row의 int_stayid가 null 값이라면, ext_stayid로 대체.
        else:
            oasis_idx = oasis_df[oasis_df["stay_id"] == row["ext_stayid"]].first_valid_index()
            
            group.loc[idx, "oasis"] = oasis_df.loc[oasis_idx, "oasis"]
            
    return group

In [8]:
df_oasis = df2.groupby(["subject_id", "hadm_id"], group_keys=False).apply(group_df_oasis)
df_oasis

,subject_id,hadm_id,int_stayid,admittime,intubationtime,int_itemid,int_weight,ext_stayid,extubationtime,ext_itemid,ext_weight,extubationcause,dischtime,deathtime,marker,mvtime,reintubation_eventtime,reintubationtime,oasis
0,10001884,26184834,37510196.0,2131-01-07 20:39:00,2131-01-11 04:30:00,224385.0,65.0,37510196.0,2131-01-12 17:40:00,227194.0,65.0,Planned Extubation,2131-01-20 05:15:00,2131-01-20 05:15:00,NaN,2230.0,NaN,NaN,34.0
1,10003400,23559586,38383343.0,2137-08-04 00:07:00,2137-08-17 21:21:00,224385.0,90.5,38383343.0,2137-08-21 15:40:00,227194.0,99.6,Planned Extubation,2137-09-02 17:05:00,2137-09-02 17:05:00,NaN,5419.0,NaN,NaN,60.0
2,10004401,27939719,31202136.0,2144-04-11 03:31:00,2144-04-11 05:03:00,224385.0,120.0,NaN,NaN,NaN,NaN,NaN,2144-04-13 17:31:00,NaN,NaN,NaN,NaN,NaN,53.0
3,10004401,29988601,32773003.0,2144-01-23 07:58:00,2144-01-27 19:00:00,224385.0,76.0,32773003.0,2144-01-30 12:30:00,227194.0,76.0,Planned Extubation,2144-02-06 11:45:00,NaN,NaN,3930.0,2144-01-30 13:00:00,30.0,46.0
4,10004401,29988601,32773003.0,2144-01-23 07:58:00,2144-01-30 13:00:00,224385.0,76.0,NaN,NaN,NaN,NaN,NaN,2144-02-06 11:45:00,NaN,NaN,NaN,NaN,NaN,46.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9003,19997367,20617667,35616526.0,2126-04-20 07:15:00,2126-04-26 20:14:00,224385.0,59.0,35616526.0,2126-05-02 16:30:00,227194.0,59.0,Planned Extubation,2126-05-19 14:15:00,NaN,NaN,8416.0,2126-05-05 18:20:00,4430.0,43.0
9004,19997367,20617667,35616526.0,2126-04-20 07:15:00,2126-05-05 18:20:00,224385.0,59.0,35616526.0,2126-05-08 17:29:00,227194.0,59.0,Planned Extubation,2126-05-19 14:15:00,NaN,NaN,4269.0,NaN,NaN,43.0
9005,19999068,21606769,30143796.0,2161-08-24 04:10:00,2161-08-25 15:34:00,224385.0,55.8,30143796.0,2161-08-28 13:35:00,227194.0,55.8,Planned Extubation,2161-09-02 19:00:00,NaN,NaN,4201.0,NaN,NaN,46.0
9006,19999442,26785317,32336619.0,2148-11-19 10:00:00,2148-11-19 19:00:00,224385.0,107.5,32336619.0,2148-11-20 14:15:00,227194.0,107.5,Planned Extubation,2148-12-04 16:25:00,NaN,NaN,1155.0,NaN,NaN,25.0


In [10]:
df_oasis["oasis"].isna().sum()

0

# 내용 정리
* Oasis 추출 
* 함수를 사용하여 데이터프레임에 붙여준다.
* 결측치는 없는 것으로 확인됨

## 문의사항

1. oasis df를 보니, column이 oasis 말고도 많은데, 추가로 병합을 진행할 컬럼이 있을까요?